# PC - Arduino serial 통신 확인

In [ ]:
# Serial 통신 with python 통신 확인
import serial

ser = serial.Serial(port = 'COM24',baudrate = 115200)

while True:
    if ser.readable(): #뭔가 읽을 수 있는 상태이면
        res = ser.readline() # ser 에서 읽어온 데이터 값 저장
        test = res.decode()[:len(res)-1]
        print(test) # res의 길이 만큼 decode 해서 프린트 함.

In [ ]:
# 통신 확인 후 통신 안전하게 닫기
if ser.isOpen():
    ser.close()

# 근전도 Gripper 코드 실행

In [ ]:
#####################################################
##################              #####################
##################     수정     #####################
##################              #####################
#####################################################

port_name = 'COM24' # 아두이노 연결 port
threshold = 100     # sEMG signal의 동작 임계치 (이 수치 이상에서 gripyper 작동)
mvc = 800           # maximum value of sEMG (이 수치 이상에서 sEMG failure 발생)
baseline1 = 0       # 첫번째 sEMG의 baseline, (input value) - (baseline) 값을 신호로 사용
baseline2 = 0       # 두번째 sEMG의 baseline, (input value) - (baseline) 값을 신호로 사용 

#####################################################
#####################################################
#####################################################

import numpy as np
import cv2
import serial
import time
import random

def rect_properties():
    global rect_width
    global rect_height
    global rect_mass
    global rect_pos_x
    global rect_pos_y
    global rect_vel_y
    global rect_acc_y   
    global rect_pos_xp
    global rect_vel_xp
    
    rect_pos_x = 500
    rect_pos_y = np.ones([2,1])*500
    rect_vel_y = 0
    rect_acc_y = g 
    
    rect_width = 80
    rect_height = 80
    rect_mass = 1
    rect_pos_xp = 0
    rect_vel_xp = 0

def img_properties():
    global img_width
    global img_height
    global img
    
    img_width = 1000
    img_height = 800
    img = np.zeros((img_height, img_width, 3), np.uint8)
    img.fill(255)

def colors_properties():
    global blue_color
    global green_color
    global red_color
    global white_color
    global brown_color
    global black_color
    global grey_color
    global magenta_color
    
    blue_color = (255, 0, 0)
    green_color = (0, 255, 0)
    red_color = (0, 0, 255)
    white_color = (255, 255, 255)
    brown_color = (0, 25, 51)
    black_color = (0,0,0)
    grey_color = (204, 204, 204)
    magenta_color = (255, 0, 255)
    
def ground_properties():
    global ground_width
    global ground_height
    global ground_stiffness
    global ground_damping
    
    ground_width = 1000
    ground_height = 50
    ground_stiffness = 100000 # k
    ground_damping = 1000 # d
    
def gripper_properties():
    global gripper_radius
    global gripper_dist_x
    global gripper_pos_y
    global gripper_stiffness
    global gripper_damping
    global gripper_penet_pos_x
    global gripper_penet_vel_x
    global gripper_direction
    
    gripper_radius = 30
    gripper_dist_x = 100 # 화면 가운데 중심으로 얼만큼 떨어져 있는지 나타낸 것
    gripper_pos_y = (img_height - (ground_height + gripper_radius))
    gripper_direction = np.zeros([2,1])
    gripper_penet_pos_x = np.zeros([2,1])
    gripper_penet_vel_x = 0
    gripper_stiffness = 1000
    gripper_damping = 0
    
def define_properties():
    global g
    global time_step
    global static_f_coefficient
    global dynamic_f_coefficient
    
    g = 9810 # [mm/s^2]
    time_step = 0.001
    static_f_coefficient = 1
    dynamic_f_coefficient = 0.5
    
    rect_properties()
    img_properties()
    colors_properties()
    ground_properties()
    gripper_properties()
    
def sEMG_control(sensor_value, threshold):
    count = 0
    if sensor_value >= threshold:
        count = -1
    else:
        count = 1
        
    return count

def cal_penet_x(sensor_value, threshold, mvc):
    max_penet = rect_width/4
    
    if (sensor_value > threshold):
        penet_x = ((max_penet / (mvc-threshold))*(sensor_value - threshold))   
    else:
        penet_x = 0
    return penet_x

def cal_f_friction(f_normal):
    global is_static
    if gripper_direction[1,0]-gripper_direction[0,0]==0 :
        f_friction = 0
        is_static = False
    else:
        static_f = 2*static_f_coefficient*abs(f_normal)
        if (static_f < abs(rect_mass*g)):
            f_friction = 2*dynamic_f_coefficient*abs(f_normal)*1
            is_static = False
        else :
            f_friction = abs(rect_mass*g)       
            is_static = True
    
        if gripper_direction[1,0]-gripper_direction[0,0] < 0:
            f_friction = abs(f_friction)
        else :
            f_friction = (-1)*abs(f_friction)
            
    return f_friction

global threshold
global mvc
global sEMG_recording1
global sEMG_recording2
global sEMG_failed

sEMG_failed = False

sEMG_recording1 = np.array([0])
sEMG_recording2 = np.array([0])
sensor1_value = 0
sensor2_value = 0

################################################################################################################################
# 전역변수들 한 번에 선언하기
define_properties()

sEMG_failure = random.randint(mvc*(4/5), mvc)

init_flag1 = 1

ser = serial.Serial(port_name, 115200)
f_normal = 0
sEMG_input = False
measure_start = False
mission_complete = False
while_step = 0

while True:   
    while_step += 1
    
    sEMG_input = False
    #start = time.time()
    
    # 아두이노에서 sEMG 신호 받기
    if ser.readable():
        ser_Arduino = ser.readline() # 아두이노에서 serial 통신으로 보낸 값 받기
        code = ser_Arduino.decode('cp1252')[:len(ser_Arduino)-1]

        # decoding
        if (len(code)>=5) and (code[0] == "<") and (len(code)<=10) :

            # decoding rule: <val1,val2>
            start_char = "<"        # starting character, 시작 문자
            end_char = ">"          # ending character, 끝 문자
            sep_char = ","          # seperater character, 데이터간 분리 문자

            start_sign = False      # starting character 수신 여부
            comma_sign = False      # ending character 수신 여부
            end_sign = False        # seperater character 수신 여부
            count = 0               # checksum
            
            for index in range(0, len(code)-1):
                if (code[index] == start_char) :
                    count = count+1    
                    start_sign = True
                if (code[index] == sep_char):
                    count = count+1
                    comma_index = index    
                    comma_sign = True
                if (code[index] == end_char):
                    count = count+1
                    end_index = index-1   
                    end_sign = True
                    break

            # 통신 수신 및 검증 확인 후 유효한 신호라면 신호 값 업데이트       
            if (count == 3) and start_sign and comma_sign and end_sign :
                sensor1 = code[1:comma_index]
                sensor2 = code[comma_index+1 : end_index+1]
                
                if (len(sensor1) <= 3)and(len(sensor2)<=3):

                    for index in range(0, len(sensor1)):
                        if (ord(sensor1[index]) < 48) or (ord(sensor1[index])>57):
                            break
                        else :
                            sensor1_value = abs(int(sensor1)-baseline1)
                            sEMG_input = True

                    for index in range(0, len(sensor2)):
                        if (ord(sensor2[index]) < 48) or (ord(sensor2[index])>57):
                            break
                        else :
                            sensor2_value = abs(int(sensor2)-baseline2)
                            sEMG_input = True
                    #print(sensor1, sensor2)
    
    #end = time.time()    
    img_temp = img.copy()
    
    #####################
    ### Object에 관한 것(물리엔진 공식)
    #####################
    
    gripper_direction[1,0] = gripper_direction[0,0]
    gripper_direction[0,0] = gripper_pos_y
    
    # 1. 땅바닥에 부딪힐 때
    if (rect_pos_y[0,0]+rect_height/2)-(img_height-ground_height) > 0 :
        rect_pos_xp = (rect_pos_y[0,0]+rect_height/2)-(img_height-ground_height)
        rect_vel_xp = rect_vel_y
        f_ground = -ground_stiffness*(rect_pos_xp)-ground_damping*(rect_vel_xp)
    else :
        f_ground = 0
    
    f_normal = (gripper_stiffness*gripper_penet_pos_x[0,0]+gripper_damping*gripper_penet_vel_x)       
    
    # calculate f_friction
    if gripper_penet_pos_x[0,0] > 1:
        is_static = False
        f_friction = cal_f_friction(f_normal)/10
    else:
        f_friction = 0
        is_static = False
    
    #f_y = (f_ground + rect_mass*g)
    
    if is_static == True :
        if (gripper_pos_y <= (rect_pos_y[0,0]+rect_height/2))and(gripper_pos_y >= (rect_pos_y[0,0]-rect_height/2)):
            rect_pos_y[1,0] = rect_pos_y[0,0]
            rect_pos_y[0,0] = gripper_pos_y
            rect_vel_y = (rect_pos_y[0,0]-rect_pos_y[1,0])/time_step   
    else:
        f_y = ((f_ground) + f_friction + rect_mass*g)

        # 3. 공중에 떠 있을 때 mass dynamics
        new_rect_pos_y = ((f_y/rect_mass))*np.power(time_step,2)+(2*rect_pos_y[0,0])-rect_pos_y[1,0]
        new_rect_vel_y = ((f_y/rect_mass))*time_step+rect_vel_y

        rect_pos_y[1,0] = rect_pos_y[0,0]
        rect_pos_y[0,0] = new_rect_pos_y
        rect_vel_y = new_rect_vel_y
        rect_acc_y = ((f_y/rect_mass))
    

    ###################
    ### Gripper
    ###################
    if sEMG_input == True:
        if (gripper_dist_x > (rect_width/2 + gripper_radius)) and (gripper_dist_x < img_width/2):
            # gripper가 화면, 사각형에 닿아있지 않다면 sEMG_control 받기
            gripper_dist_x = gripper_dist_x + sEMG_control(sensor1_value, threshold)
        elif (gripper_dist_x == rect_width/2 + gripper_radius):
            # gripper가 사각형에 닿아있다면
            if (sensor1_value >= threshold) :
                # sEMG 신호가 gripper를 움직이려고 하면, 사각형이 눌린 거리 계산
                if init_flag1 == 1 :
                    init_flag1 = 0
                    # 여기서부터 sEMG effort 얼마나 들어갔는지 기록
                else :    
                    gripper_dist_x = gripper_dist_x

                    if (gripper_pos_y <= (rect_pos_y[0,0]+rect_height/2))and(gripper_pos_y >= (rect_pos_y[0,0]-rect_height/2)):
                        gripper_penet_pos_x[1,0] = gripper_penet_pos_x[0,0]
                        gripper_penet_pos_x[0,0] = cal_penet_x(sensor1_value, threshold, mvc)
            else :
                gripper_dist_x = gripper_dist_x +1
        elif gripper_dist_x >= img_width/2 :
            # gripper가 화면 밖으로 나가려고 하면
            gripper_dist_x = (img_width/2)-1
            gripper_penet_pos_x[1,0] = gripper_penet_pos_x[0,0]   
            gripper_penet_pos_x[0,0] = 0
        
        if (gripper_pos_y < (img_height - (ground_height+gripper_radius))) and (gripper_pos_y > gripper_radius):            
            gripper_pos_y = gripper_pos_y + sEMG_control(sensor2_value, threshold)               
        elif (gripper_pos_y == img_height - (ground_height + gripper_radius)):
            if (sensor2_value < threshold):
                gripper_pos_y = gripper_pos_y
            else :
                gripper_pos_y = gripper_pos_y-1
        elif (gripper_pos_y == gripper_radius):
            gripper_pos_y = gripper_radius +1

    # 화면 Update 주기: while문이 10번 반복될때마다 1번씩 화면을 새로고침함.
    if (while_step % 10 == 0):
        # record sEMG_input - sEMG value가 threshold 넘었을 때 그 차이만큼 기록
        if sensor1_value >= threshold:
            sEMG_recording1 = np.append(sEMG_recording1 , np.array([sensor1_value]))
        if sensor2_value >= threshold:
            sEMG_recording2 = np.append(sEMG_recording2 , np.array([sensor2_value]))
        
        
        # showing gripper
        cv2.circle(img_temp, (int(img_width/2 - gripper_dist_x), int(gripper_pos_y)), gripper_radius, blue_color, -1)    
        cv2.circle(img_temp, (int(img_width/2 + gripper_dist_x), int(gripper_pos_y)), gripper_radius, blue_color, -1)

        # showing ground
        cv2.rectangle(img_temp, (0,int(img_height-ground_height)), (int(img_width), int(img_height)), brown_color, -1)
        
        # showing object    
        cv2.rectangle(img_temp, (int(rect_pos_x-rect_width/2), int(rect_pos_y[0,0] - rect_height/2)), (int(rect_pos_x+rect_width/2), int(rect_pos_y[0,0]+rect_height/2)), green_color, -1)
        
        if measure_start :
            #show text
            cv2.putText(img_temp, str(sensor1_value), (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, red_color, 2)
            cv2.putText(img_temp, ",", (120,100), cv2.FONT_HERSHEY_SIMPLEX, 1, red_color, 2)
            cv2.putText(img_temp, str(sensor2_value), (130,100), cv2.FONT_HERSHEY_SIMPLEX, 1, red_color, 2)
            
            # showing goal line
            cv2.putText(img_temp,"GOAL LINE", (415, 135), cv2.FONT_HERSHEY_SIMPLEX, 1, red_color, 2)
            cv2.line(img_temp, (0, 150), (img_width, 150), red_color, 2)
            
            # showing sEMG value bar
            bar_max = 200
            bar_min = 700
            
            cv2.rectangle(img_temp, (940, bar_max), (950, bar_min), grey_color, -1)
            cv2.rectangle(img_temp, (880, bar_max), (890, bar_min), grey_color, -1)
            
            bar_mvc = 250            
            bar_threshold = bar_min - int((bar_min-bar_mvc)*(threshold/mvc))
            bar_failure = bar_min - int((bar_min-bar_mvc)*(sEMG_failure/mvc))
            
            sEMG_center1 = bar_min - int((bar_min-bar_mvc)*(sensor1_value/mvc))
            sEMG_center2 = bar_min - int((bar_min-bar_mvc)*(sensor2_value/mvc))
            
            if sEMG_center1 <= bar_max:
                sEMG_center1 = bar_max
            if sEMG_center2 <= bar_max:
                sEMG_center2 = bar_max
            
            cv2.rectangle(img_temp, (940, bar_failure), (950, bar_failure+5), red_color, -1)
            cv2.rectangle(img_temp, (880, bar_failure), (890, bar_failure+5), red_color, -1)
            
            cv2.rectangle(img_temp, (940, bar_threshold), (950, bar_threshold+5), red_color, -1)
            cv2.rectangle(img_temp, (880, bar_threshold), (890, bar_threshold+5), red_color, -1)
            
            cv2.putText(img_temp, "threshold", (800, bar_threshold + 7), cv2.FONT_HERSHEY_SIMPLEX, 0.5, black_color, 1)
            cv2.putText(img_temp, "sEMG_limit", (790, bar_failure + 7), cv2.FONT_HERSHEY_SIMPLEX, 0.5, black_color, 1)
            
            cv2.putText(img_temp, "sEMG1" , (857, bar_max-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, black_color, 1)
            cv2.putText(img_temp, "sEMG2" , (917, bar_max-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, black_color, 1)
            
            cv2.circle(img_temp, (885, sEMG_center1), 10, magenta_color, -1)
            cv2.circle(img_temp, (945, sEMG_center2), 10, magenta_color, -1)
            
            # sEMG 초과로 인한 실패 판단
            if (sEMG_center1 < bar_failure) or (sEMG_center2 < bar_failure):
                sEMG_failed = True
            
            # 미션 성공 여부 판단
            if rect_pos_y[0,0] <= 135 :
                mission_complete = True
                time_end = time.time()
            
            #cv2.putText(img_temp, str(rect_pos_y),(50,150), cv2.FONT_HERSHEY_SIMPLEX, 1, red_color, 2)
            #cv2.putText(img_temp, str(f_y), (50,200), cv2.FONT_HERSHEY_SIMPLEX, 1, red_color, 2)
        else : # 초기 조건 - 시작 화면
            cv2.putText(img_temp, "Press y/Y to start", (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, black_color, 2)
            cv2.putText(img_temp, "Total Score : 1500 - sEMG value - (time duration [sec])*10", (50,150), cv2.FONT_HERSHEY_SIMPLEX, 0.5, black_color, 1)
            if cv2.waitKey(0) == ord('y'):
                measure_start = True      
                time_start = time.time()

        # q 키를 누르면 종료하게 하기, 
        if cv2.waitKey(1) == ord('q'):
            break
        
        # Calculate Score
        if mission_complete :
            score = (np.mean(sEMG_recording1)+np.mean(sEMG_recording2))
            total_score = 1500 - score - (time_end - time_start)*10
            if total_score < 0 :
                total_score = 0

            cv2.putText(img_temp, "sEMG = ", (50,400), cv2.FONT_HERSHEY_SIMPLEX, 1, black_color, 2) 
            cv2.putText(img_temp, str(score), (200,400), cv2.FONT_HERSHEY_SIMPLEX, 1, red_color, 2) 
            cv2.putText(img_temp, "Time = ", (50,450), cv2.FONT_HERSHEY_SIMPLEX, 1, black_color, 2) 
            cv2.putText(img_temp, str(time_end - time_start), (200,450), cv2.FONT_HERSHEY_SIMPLEX, 1, red_color, 2) 

            cv2.putText(img_temp, "Total Score = ", (50,550), cv2.FONT_HERSHEY_SIMPLEX, 1, black_color, 2) 
            cv2.putText(img_temp, str(total_score), (300,550), cv2.FONT_HERSHEY_SIMPLEX, 1, red_color, 2)
            cv2.putText(img_temp, "(Press any key to RESTART(take few seconds), q to QUIT)", (50,600), cv2.FONT_HERSHEY_SIMPLEX, 1, black_color, 2) 
            cv2.imshow("image_temp", img_temp)

            if cv2.waitKey() == ord('q'): break
            else:
                mission_complete = False
                continue
        
        # Calculate Score
        if sEMG_failed == True:
            cv2.putText(img_temp, "sEMG value is too high !!!!!!", (50,400), cv2.FONT_HERSHEY_SIMPLEX, 1, black_color, 2)
            cv2.putText(img_temp, "Total Score = ", (50,550), cv2.FONT_HERSHEY_SIMPLEX, 1, black_color, 2) 
            cv2.putText(img_temp, str(0), (300,550), cv2.FONT_HERSHEY_SIMPLEX, 1, red_color, 2)
            cv2.putText(img_temp, "(Press any key to RESTART(take few seconds), q to QUIT)", (50,600), cv2.FONT_HERSHEY_SIMPLEX, 1, black_color, 2) 
            cv2.imshow("image_temp", img_temp)

            key_cmd = cv2.waitKey()
            if key_cmd == ord('q'): break
            else:   
                ser.close() 
                ser = serial.Serial(port_name, 115200)
                sEMG_failed = False
                continue
        
        # 화면 업데이트
        cv2.imshow("image_temp", img_temp)

cv2.imshow("image_temp", img_temp)
ser.close()
cv2.destroyAllWindows()